# Basic depiction of a FEP workflow using BioSimSpace and Python

In [1]:
import BioSimSpace as BSS
import glob

BSS.__version__


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



'2020.1.0+58.g817f66f'

## 1. Load ligands and protein into memory.

Loading ligands and protein structures can be done using BSS API.

In [12]:
path_to_protein = "inputs/protein/ROS1.pdb"
path_to_ligands = "inputs/ligands/*.mol2"

# load the protein.
protein = BSS.IO.readPDB(path_to_protein)

# loading all the ligands is a bit more involved. The simplest way is to glob over all ligand paths
# and load them into a dictionary so that we can pick molecules when we set up edges later on.
ligands_dict = {}
for ligand_path in glob.glob(path_to_ligands):
    
    # first, derive the ligand name.
    ligand_name = ligand_path.split("/")[-1].replace(".mol2", "")
    
    # make an entry in our dictionary with the ligand name and the loaded molecule.
    ligands_dict[ligand_name] = BSS.IO.readMolecules(ligand_path)
    
print("Number of loaded ligands:", len(ligands_dict))

Number of loaded ligands: 20


## 2. Parameterise protein and ligands.


In [4]:
# parameterise the proteins with a force field of choice. We'll go with ff99SB.
protein_p = BSS.Parameters.ff99SB(protein)

# parameterise the ligands. To keep it clean we'll just take each ligand key, and 
# replace its value molecule by the parameterised version.
for ligand_name, molecule in ligands_dict.items():
    
    # replace the value molecule in the dict with the parameterised version.
    molecule_p = BSS.Parameters.gaff2(molecule)
    ligands_dict[ligand_name] = molecule_p


## 3. Determine which edges to run using LOMAP.


In [5]:
# here we would do either manual edge selection or using LOMAP (link to github). For our purposes we'll 
# use a set of pre-selected edges.

## tmp way of selecting edges is by checking out the freenrgworkflows output file that we have on ROS1.
transformations = []
with open("outputs/summary.csv", "r") as summaryfile:
    for row in summaryfile:
        if row.startswith("ROS"):
            pert = row.split(",")[:2]
            transformations.append(pert)


## 4. Set up SOMD simulation folders.


In [12]:
# Define the Free Energy protocol. Here we use BSS's standard settings.
# see https://biosimspace.org/api/generated/BioSimSpace.Protocol.FreeEnergy.html#BioSimSpace.Protocol.FreeEnergy 
protocol = BSS.Protocol.FreeEnergy()

In [13]:
# We can loop over each transformation and do the standard BSS setup protocol.
for pert in transformations:
    # derive the perturbation name (to name our simulation folder).
    pert_name = pert[0]+"~"+pert[1]
    
    # get the molecule objects from our dictionary.
    ligand_A = ligands_dict[pert[0]]
    ligand_B = ligands_dict[pert[1]]
    
    # generate a mapping between the two molecules.
    mapping = BSS.Align.matchAtoms(ligand_A, ligand_B)
    
    # align ligand A to ligand B.
    ligand_A_a = BSS.Align.rmsdAlign(ligand_A, ligand_B, mapping)
    
    # merge the aligned molecules into a single object.
    merged = BSS.Align.merge(ligand_A_a, ligand_B, mapping,
                                        allow_ring_breaking=True,
                                        allow_ring_size_change=True
                                        )
    
    # insert the merged molecules into the protein and solvate the system.
    system = protein_p + merged
    system_s = BSS.Solvent.tip3p(molecule=system, box=3*[10*BSS.Units.Length.nanometer])
    
    # set up a SOMD folder with standard settings.
    # if you want to run replicates here, a simple iteration loop that with work_dir=pert_name+str(i) would work.
    BSS.FreeEnergy.Binding(
                                            system_s, 
                                            protocol, 
                                            engine="SOMD",
                                            work_dir=pert_name
                                            )

## 5. Run simulations & gather MBAR results from simulation outputs.

The collection of set up SOMD folders can be run in parallel on e.g. a computing cluster with slurm. An example job script for such a system would like this:

In [14]:
## insert slurm script here. Preferably this will be a script that loads the SOMD env, 
# does freenrg.run() and freenrg.analyse() (does analyse write out freenrg-MBAR.dat files?)

# if that implementation is too complex, I always run ~/biosimspace.app/bin/somd-freenrg and subsequently mbar
# directly in each lambda folder. 

## 6. Compute statistics and plot validations.

In [ ]:
# use freenrgworkflows? might be easy to build into BSS. Otherwise a standard set of functions using basic plt
# could work too. 